# VacationPy
--- 

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [47]:
# Dependencies and Setup
import os
import pandas as pd
import hvplot.pandas
import geoviews as gv
from pathlib import Path
import requests

# Import API key
from api_keys import geoapify_key
# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

In [48]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
input_path = os.path.join("..","output_data","cities.csv")
city_data_df = pd.read_csv(input_path,index_col="City_ID")


#create a dataframe from the city data
city_data_df = pd.DataFrame(city_data_df)

#Edit city names to start from a capital letter - source: ChatGPT 
city_data_df["City"] = city_data_df["City"].str.title()

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Time
City_ID,,,,,,,,,,
0,Serowe,-22.3833,26.7167,16.79,29,0,1.71,BW,2023-06-05,18:30:00
1,Sindgi,16.9167,76.2333,28.81,54,56,8.61,IN,2023-06-05,18:30:00
2,Mount Gambier,-37.8333,140.7667,14.17,84,34,4.77,AU,2023-06-05,18:30:00
3,Jamestown,42.0970,-79.2353,20.01,45,0,2.06,US,2023-06-05,18:25:23
4,Grytviken,-54.2811,-36.5092,2.36,97,100,2.04,GS,2023-06-05,18:30:00


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display
#Source: 06.03.06-Evr_Geoviews_Maps
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500, 
    color='City'
    )
    
# Display the map plot_1
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [50]:
# Narrow down cities that fit criteria and drop any results with null values
# narrow down the dataframe to cities with lat > 0, max temp between 20 and 30 degrees and wind speed less than 10 m/s and zero cloudiness
#Source: 04.02.05-Evr_Pandas_Recap & 04.02.09-Stu_SearchForTheWorst
perfect_cities_df = city_data_df.loc[(city_data_df["Lat"] > 0) & (city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 30) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
#Source: 04.02.03-Ins_CleaningData
perfect_cities_df = perfect_cities_df.dropna(how='any')

# Display sample data
perfect_cities_df.head(30)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Time
City_ID,,,,,,,,,,
3,Jamestown,42.0970,-79.2353,20.01,45,0,2.06,US,2023-06-05,18:25:23
11,Mount Pleasant,32.7941,-79.8626,25.66,68,0,2.57,US,2023-06-05,18:26:11
19,Kill Devil Hills,36.0307,-75.6760,21.00,74,0,3.09,US,2023-06-05,18:30:04
26,Wailua Homesteads,22.0669,-159.3780,26.59,72,0,4.12,US,2023-06-05,18:30:06
51,Tamanrasset,22.7850,5.5228,23.95,16,0,2.06,DZ,2023-06-05,18:30:10
57,Tyukhtet,56.5389,89.3011,21.97,61,0,0.92,RU,2023-06-05,18:30:12
91,Remire-Montjoly,4.9167,-52.2667,27.02,83,0,1.03,GF,2023-06-05,18:30:17
104,Northport,33.2290,-87.5772,26.90,65,0,1.54,US,2023-06-05,18:30:19
130,Hamilton,39.1834,-84.5333,27.12,33,0,4.63,US,2023-06-05,18:26:56


### Step 3: Create a new DataFrame called `hotel_df`.

In [51]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
#source:https://stackoverflow.com/questions/34682828/extracting-specific-selected-columns-to-new-dataframe-as-a-copy
hotel_df = perfect_cities_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
3,Jamestown,US,42.0970,-79.2353,45,
11,Mount Pleasant,US,32.7941,-79.8626,68,
19,Kill Devil Hills,US,36.0307,-75.6760,74,
26,Wailua Homesteads,US,22.0669,-159.3780,72,
51,Tamanrasset,DZ,22.7850,5.5228,16,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [52]:
# Set parameters to search for a hotel
#Source 06.03.05-Evr_Exploring_Airports
radius = 50000
params = {
    "categories": "accommodation",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Jamestown - nearest hotel: DoubleTree Jamestown
Mount Pleasant - nearest hotel: Old Village Post House
Kill Devil Hills - nearest hotel: Mariner Days Inn & Suites
Wailua Homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
Tamanrasset - nearest hotel: فندق الأمان
Tyukhtet - nearest hotel: No hotel found
Remire-Montjoly - nearest hotel: Motel du Lac
Northport - nearest hotel: Best Western Catalina Inn
Hamilton - nearest hotel: North Vista Manor
Praia - nearest hotel: Casa Sodadi
Tomatlan - nearest hotel: Hotel Hacienda Vieja Tomatlan
Lihue - nearest hotel: Kauai Palms
Kapa'A - nearest hotel: Pono Kai Resort
Alice - nearest hotel: No hotel found
Altay - nearest hotel: Sunshine Holiday Hotel
Tura - nearest hotel: Eco Park Rest House
Laguna - nearest hotel: Holiday Inn Express & Suites
Saint-Aubin-De-Medoc - nearest hotel: Kyriad Direct
Fort Bragg - nearest hotel: Airborne Inn Lodging
Puerto Real - nearest hotel: Hostal Bahía de Cádiz
Pontchateau - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
3,Jamestown,US,42.0970,-79.2353,45,DoubleTree Jamestown
11,Mount Pleasant,US,32.7941,-79.8626,68,Old Village Post House
19,Kill Devil Hills,US,36.0307,-75.6760,74,Mariner Days Inn & Suites
26,Wailua Homesteads,US,22.0669,-159.3780,72,Hilton Garden Inn Kauai Wailua Bay
51,Tamanrasset,DZ,22.7850,5.5228,16,فندق الأمان
57,Tyukhtet,RU,56.5389,89.3011,61,No hotel found
91,Remire-Montjoly,GF,4.9167,-52.2667,83,Motel du Lac
104,Northport,US,33.2290,-87.5772,65,Best Western Catalina Inn
130,Hamilton,US,39.1834,-84.5333,33,North Vista Manor


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

#Configure the map plot
#source hover message: https://stackoverflow.com/questions/59678780/show-extra-columns-when-hovering-in-a-scatter-plot-with-hvplot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM',
    frame_width = 700,
    frame_height = 500, 
    color='City',
    hover_cols=["Hotel Name","Country"]
    )
    
# Display the map plot_1
map_plot_2



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)